# assignment 2
Start by uploading the datafile to Databricks. Use later the search path to the file to the file in the next cell.

In [ ]:
pip install pyspark

In [68]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("KDDCupAnalysis").getOrCreate()
data = spark.read.csv("kddcup.data_10_percent_corrected", inferSchema=True, header=True)
data.createOrReplaceTempView('data')


In [69]:
# Count the number of rows in the data
data.count()

494021

In [70]:
# Visa de första 20 raderna
data.show(20)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+----------------+-----------+-------------+---------+-------------------+-----------+-----------------+------------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|lnum_compromised|lroot_shell|lsu_attempted|lnum_root|lnum_file_creations|lnum_shells|lnum_access_files|lnum_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_h

In [71]:
# Här är ett exempel på en sökning i datan
interactions = spark.sql("""
SELECT duration, dst_bytes FROM data WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [72]:
# Vi kan använda printSchema() för att visa vilka kolumner det finns
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- lnum_compromised: integer (nullable = true)
 |-- lroot_shell: integer (nullable = true)
 |-- lsu_attempted: integer (nullable = true)
 |-- lnum_root: integer (nullable = true)
 |-- lnum_file_creations: integer (nullable = true)
 |-- lnum_shells: integer (nullable = true)
 |-- lnum_access_files: integer (nullable = true)
 |-- lnum_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullabl

In [73]:
# Exempel på gruppering av data (Spark-syntax, ej SQL-syntax)
data.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+



In [74]:
data.select("protocol_type", "duration", "dst_bytes").filter(data.duration>1000).filter(data.dst_bytes==0).groupBy("protocol_type").count().show()

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+



## Uppgifter
Skriv koden för era lösningar i celler här nedanför

**a.Which protocols are represented in the data?**

In [75]:
protocols = data.select("protocol_type").distinct()
protocols.show()


+-------------+
|protocol_type|
+-------------+
|          tcp|
|          udp|
|         icmp|
+-------------+



**b.How many services are using the protocol..**

In [76]:
# Count services by protocol (assuming column 3 is 'service')
service_count = data.groupBy("protocol_type", "service").count()
service_count.show()


+-------------+----------+-----+
|protocol_type|   service|count|
+-------------+----------+-----+
|         icmp|     eco_i| 1642|
|          tcp|       mtp|  107|
|          tcp|  csnet_ns|  126|
|          tcp|   pm_dump|    1|
|          tcp|    finger|  670|
|          udp|  domain_u| 5863|
|          tcp|       IRC|   43|
|          tcp|      time|  157|
|          tcp|      link|  102|
|          tcp|    kshell|   98|
|          tcp| uucp_path|  106|
|          tcp|netbios_ns|  102|
|          tcp|    gopher|  117|
|          tcp|     whois|  110|
|          tcp|       ctf|   97|
|          tcp|   netstat|   95|
|          tcp|      name|   98|
|          tcp|      auth|  328|
|          tcp|remote_job|  120|
|          tcp|   sql_net|  110|
+-------------+----------+-----+
only showing top 20 rows



**How many packets are classified as part of an attack, and using the protocol…**

In [77]:
attack_packets = data.filter(data["label"] != "normal.")
attack_packets.groupBy("protocol_type").count().show()


+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|113252|
|          udp|  1177|
|         icmp|282314|
+-------------+------+



**What percentage of the total number of packages (rounded to whole percentages) has a label that is not "normal"?**

In [78]:
total_packets = data.count()
print("Total data packets: ",total_packets)

total_attack_packets = attack_packets.count()
print("Total attacked data packets: ",total_attack_packets)

attack_percentage = (total_attack_packets / total_packets) * 100
print(f"Percentage of attack packets: {round(attack_percentage)}%")


Total data packets:  494021
Total attacked data packets:  396743
Percentage of attack packets: 80%


**How many ICMP packets are classified as part of an attack, but are not a so-called "smurf" attack?**

In [79]:
icmp_attack_packets = attack_packets.filter(attack_packets["protocol_type"] == "icmp")

# Exclude 'smurf' attacks
non_smurf_icmp_attacks = icmp_attack_packets.filter(icmp_attack_packets["label"] != "smurf")
non_smurf_icmp_attacks_count = non_smurf_icmp_attacks.count()
print(f"Non-smurf ICMP attack packets: {non_smurf_icmp_attacks_count}")


Non-smurf ICMP attack packets: 282314
